In [ ]:
# Copyright 2019 Google Inc. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [ ]:
!PIP_DISABLE_PIP_VERSION_CHECK=1 pip3 install kfp --upgrade --user --quiet

In [ ]:
FIRSTNAME_LASTNAME = 'firstname_lastname'
EXPERIMENT_NAME = 'Hello world! - reusable components 1 ' + FIRSTNAME_LASTNAME

# Build Reusable Components

Documentation: https://www.kubeflow.org/docs/pipelines/sdk/component-development/

In [ ]:
import kfp.components as comp

In [ ]:
#Define a Python function
def add(a: float, b: float) -> float:
    print('adding ',a,' and ',b)
    return a+b

### Save Reusable Components
Using the function created above we will create a component text file

In [ ]:
with open('add_component.yaml', 'w') as file:
    file.write(comp.func_to_component_text(add))

In [ ]:
!cat add_component.yaml

We can load the component from the file

In [ ]:
add_op = comp.load_component_from_file ('add_component.yaml')

#### Define the pipeline
Pipeline function has to be decorated with the `@dsl.pipeline` decorator

In [ ]:
import kfp.dsl as dsl
@dsl.pipeline(
   name='Reusable component',
   description='A toy pipeline that demonstrates reusable components'
)
def reusable_component(
   a='a',
   b='7'
):
    #building a one step pipeline to Visualize the results 
    add_op_task = add_op(a, b)
    

#### Compile the pipeline

In [ ]:
pipeline_func = reusable_component
pipeline_filename = pipeline_func.__name__ + '.pipeline.zip'
import kfp.compiler as compiler
compiler.Compiler().compile(pipeline_func, pipeline_filename)

#### Submit the pipeline for execution

In [ ]:
#Get or create an experiment and submit a pipeline run
import kfp
client = kfp.Client()
experiment = client.create_experiment(EXPERIMENT_NAME)

In [ ]:
#Specify pipeline argument values
arguments = {'a': '7', 'b': '8'}

#Submit a pipeline run
run_name = pipeline_func.__name__ + ' ' + FIRSTNAME_LASTNAME + ' run'
run_result = client.run_pipeline(experiment.id, run_name, pipeline_filename, arguments)